In [1]:
import sys
import time
import numpy as np
from kin2d import Kin2D 
from kiddobot import Kiddobot

In [2]:
robot=Kiddobot()
robot.start_sim()

Kiddobot class
vrep simulated robot, version= March, 9, 2020
Connected to remote API server


In [3]:
# robot.set_base_elbow(45, 60)

In [4]:
robot.go_def()

0

In [5]:
robot.pen_down()

pen down to do


1

In [6]:
robot.pen_up()

pen up to do


0

In [7]:
d, d2=robot.get_joint_pos()
print(d, d2)

29.99998375935313 29.99983349519139


In [118]:
robot.close()

connection closed...


#### Kin test

In [8]:
kin=Kin2D(200, 180, verbose=False)

2D Kin. version= March, 9, 2020
Link1= 200 mm  Link2= 180 mm


In [80]:
x,y=kin.calcFK(20, 30)
print('x=',x,' y=',y)

x= 303.6402939007588  y= 206.29202842654973


In [81]:
sx = 23
sy = 146

In [82]:
s, d1, d2=kin.calcIK(sx, sy)
print(s, d1, d2)

True 20.918796232550893 134.59997785363097


In [83]:
def go_to_xy(x, y):
    s, d1, d2=kin.calcIK(x, y)
    if not s:
        return False
    else:
        robot.set_base_elbow(d1, d2)
    return True

In [84]:
# s, d1, d2=kin.calcIK(12121, 43)
# print(s,d1, d2)

In [85]:
go_to_xy(sx, sy)

True

In [86]:
robot.pen_down()

pen down to do


0

In [87]:
go_to_xy(sx, sy)

True

In [88]:
robot.pen_up()

pen up to do


0

In [23]:
go_to_xy(x, y)

True

In [24]:
go_to_xy(50, 200)

True

In [25]:
go_to_xy(150, 200)

True

In [26]:
x0=50
y0=200

In [27]:
def line_eq(m,c, x):
    return m*x+c

In [28]:
npt=50
xx=np.array( [i for i in range(npt)] )
print(xx)
yy=line_eq(0.5, 0, xx)
print(yy)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5 16.  16.5 17.  17.5 18.  18.5 19.  19.5 20.  20.5
 21.  21.5 22.  22.5 23.  23.5 24.  24.5]


In [29]:
nx=x0+xx
ny=y0+yy
print(nx)
print(ny)

[50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73
 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97
 98 99]
[200.  200.5 201.  201.5 202.  202.5 203.  203.5 204.  204.5 205.  205.5
 206.  206.5 207.  207.5 208.  208.5 209.  209.5 210.  210.5 211.  211.5
 212.  212.5 213.  213.5 214.  214.5 215.  215.5 216.  216.5 217.  217.5
 218.  218.5 219.  219.5 220.  220.5 221.  221.5 222.  222.5 223.  223.5
 224.  224.5]


In [33]:
ds=[]
ds2=[]
failed=0
for ix, iy in zip(nx, ny):
    s, d1, d2=kin.calcIK(ix, iy)
    if s:
        ds.append(d1)
        ds2.append(d2)
    else:
        failed+=1
print('failed point=',failed)

failed point= 0


In [34]:
# print(ds)
# print(ds2)

In [35]:
go_to_xy(x0, y0)

True

In [36]:
robot.pen_down()
time.sleep(0.2)
for di, di2 in zip(ds, ds2):
    time.sleep(0.1)
    robot.set_base_elbow(di, di2)
    
time.sleep(0.1)
robot.pen_up()

pen down to do
pen up to do


0

### ToDo : Write Trajectory generator function

In [110]:
#y range: 150 300
#x range: -150 150 
xmin=-150
ymin=120
xd, yd=300, 200
# go_to_xy(100, 300)

In [111]:
# go_to_xy(-150, 300)
x=xmin
y=ymin
go_to_xy(x, y)

True

In [112]:
x0,y0=-150, 150
xm,ym=150, 300

In [113]:
range(x0, xm)

range(-150, 150)

In [117]:
robot.pen_up()
go_to_xy(x, y)

x=xmin
time.sleep(1)
robot.pen_down()
failed=0
for ix in range(x, x+xd):
    time.sleep(0.01) 
    s=go_to_xy(ix, y)
    if not s:
        failed+=s
    else:
        x=ix
print('failed=',failed)

pen up to do
pen down to do
failed= 0


In [115]:
# robot.pen_up()
# go_to_xy(x0,y0)

# time.sleep(1)
robot.pen_down()
failed=0
for iy in range(y, y+yd):
    time.sleep(0.01) 
    s=go_to_xy(x, iy)
    if not s:
        failed+=s
    else:
        y=iy
print('failed=',failed)

pen down to do
failed= 0


### AutoRobot

In [31]:
class AutoKiddobot():
    def __init__(self):
        print('Kiddobot sim and kin auto')
        self.robot=Kiddobot()
        self.kin=Kin2D(200, 180, verbose=False)
        self.x0= 150
        self.y0=120
        self.xdist=-300
        self.ydist=200
        self.x=0
        self.y=0
        
    def start(self):
        self.robot.start_sim()
    def go_to_xy(self, x, y):
        s, d1, d2=self.kin.calcIK(x, y)
        if not s:
            return False
        else:
            self.robot.set_base_elbow(d1, d2)
        self.x=x
        self.y=y
        return True
    def get_current_xy(self):
        d1, d2=self.robot.get_joint_pos()
        x,y=self.kin.calcFK(d1, d2)
        self.x=x
        self.y=y
        return x,y
    def pen_up(self):
        self.robot.pen_up()
    def pen_down(self):
        self.robot.pen_down()
    def close(self):
        self.robot.close()

In [32]:
bot=AutoKiddobot()
bot.start()

Kiddobot sim and kin auto
Kiddobot class
2D Kin. version= March, 9, 2020
Link1= 200 mm  Link2= 180 mm
vrep simulated robot, version= March, 9, 2020
Connected to remote API server


In [33]:
bot.go_to_xy(150, 200)

True

In [35]:
bot.go_to_xy(bot.x0, bot.y0)

True

In [38]:
bot.close()

connection closed...


In [37]:
print(bot.x, bot.y)

150 120
